In [2]:
import pandas as pd
import pprint
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import col
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import FloatType, BooleanType, StructField, StructType, DoubleType, ArrayType
import pickle
import math
import time

import os
from dotenv import load_dotenv
load_dotenv()

AWS_ENDPOINT_URL = os.getenv('AWS_ENDPOINT_URL')
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY')

In [3]:
conf = SparkConf().setAppName("Spark com S3").setMaster("local[*]")

conf.set("spark.driver.memory", "70g")
conf.set("spark.executor.memory", "70g")
conf.set("spark.executor.pyspark.memory", "70g")

# conf.set("spark.driver.cores", "20")
# conf.set("spark.executor.cores", "20")

# conf.set("spark.memory.offHeap.enabled", "true")
# conf.set("spark.memory.offHeap.size", "20g")

# conf.set("spark.sql.shuffle.partitions", "2000")
# conf.set("spark.sql.parquet.columnarReaderBatchSize", "2048") 
conf.set("spark.sql.parquet.enableVectorizedReader", "false")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.repl.eagerEval.enabled", "true")
conf.set("spark.sql.repl.eagerEval.truncate", 100)

conf.set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
conf.set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
conf.set("spark.hadoop.fs.s3a.endpoint", AWS_ENDPOINT_URL)
conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

24/08/07 13:05:54 WARN Utils: Your hostname, titan resolves to a loopback address: 127.0.1.1; using 139.99.60.146 instead (on interface eno1)
24/08/07 13:05:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/xavier/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/xavier/.ivy2/cache
The jars for the packages stored in: /home/xavier/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-29234c50-1886-4811-9f00-a94f5bf7a292;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 222ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------

In [ ]:
df = spark.read.parquet('../data/ecommerce_technologies_v2')

In [ ]:
from pyspark.sql.functions import explode, col

technologies_df = spark.read.csv('../data/technologies.csv', header=True)
technologies_list = [row.TECHNOLOGIES for row in technologies_df.collect()]

exploded_df = df.select(explode(col('ecommerce_technology')).alias('technology'))

distinct_technologies = [row.technology for row in exploded_df.select('technology').distinct().collect()]

non_matching_technologies_list = [tech for tech in technologies_list if tech not in distinct_technologies]

print(non_matching_technologies_list)

['AGENCIA', 'DEVHOUSE', 'ECCOSYS', 'KPL', 'SOFTVAR', 'FULL COMMERCE', 'GATEWAY', 'LOJA TESTE', 'MODO CALL CENTER', 'AG COMMERCE', 'AZTRONIC', 'BSELLER', 'BETALABS', 'BIS2BIS', 'BIZ COMMERCE', 'BLUEFOOT', 'COMM2', 'DIGITAL HUB', 'E-SMART', 'EZCOMMERCE', 'FBITS', 'FULL SOLUTIONS', 'GIRAN/ADENA', 'INNERSITE', 'JET ECOMMERCE', 'MAXIMAWEB', 'MOOVIN', 'NETFASHION', 'PRIMORDIA', 'SIGNATIVA', 'SMARTBILL', 'TATIX', 'TREZO', 'UECOMMERCE', 'VANNON', 'WEBJUMP', 'XTECH', 'SOFTWARE', '3 DOTS', '88 DIGITAL', 'ACCON', 'ADENA', 'AGILE COMMERCE', 'AGORA B2B', 'ALTEGIO', 'AMPLIA VENDAS', 'ANYSHOP', 'ASSINE STORE', 'AUREA', 'AXYSWEB', 'B4 COMMERCE', 'BATINGA CURSOS', 'BELL TECNOLOGIA', 'BETA SISTEMAS', 'BIG FISH', 'BIG SHOP', 'BRAAVO', 'BRAND ECOMMERCE', 'BRASIL NA WEB', 'BRING E-COMMERCE', 'BW COMMERCE', 'CAMMINO', 'CARTPANDA', 'CAZCO', 'CHEZ', 'CIASHOP', 'CLIMBA', 'COMMERCE PLUS', 'CONECTA LÁ', 'CONVERTIZE', 'CONVERTR', 'CRIAR LOJA ONLINE', 'DAWEB', 'DEZWORK', 'DIGI GIRLS', 'DIGITAL MANAGER GURU', 'DIGI

In [12]:
# Muita divergência por conta de acentos, espaços e versionamento de tecnologias (ex: 'Magento 2' e 'Magento')
len(non_matching_technologies_list)

237